## Object detection using SIFT (Not available in openCV 3.0+)

In [ ]:
import cv2
import numpy as np


def sift_detector(new_image, image_template):
    # Function that compares input image to template
    # It then returns the number of SIFT matches between them
    
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    image2 = image_template
    
    # Create SIFT detector object
    sift = cv2.SIFT()

    # Obtain the keypoints and descriptors using SIFT
    # descriptor는, 이전에 언급했듯, vectors that store information about the key points. match의 대상이 되는 애들임.
    keypoints_1, descriptors_1 = sift.detectAndCompute(image1, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(image2, None)
    
    ### Flann_Matcher 사용.
    # Define parameters for our Flann Matcher
    FLANN_INDEX_KDTREE = 0 # 초기값 0으로 설정
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3) # tree가 많을수록 complicated, slower
    search_params = dict(checks = 100) # how many matches are going to try to compute

    # Create the Flann Matcher object
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Obtain matches using K-Nearest Neighbor Method
    # the result 'matchs' is the number of similar matches found in both images
    # descriptor 1과 2가 가장 가까운 것들. (similar matches found in both img.)
    matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)

    # Store good matches using Lowe's ratio test
    # flannbasedmatcher는 정확도가 높지 않은 대신 속도가 빠르다. 정확성을 높이기 위해 사용하는 게 lowe's ratio test.
    # distance는 numpy func이고, m.distance가 기준을 통과하면 good_match에 append한다.
    good_matches = []
    for m,n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m) 

    # 결과적으로 중요한 건 'how many matches are in'이기 때문에 len() 결과값을 리턴한다.
    return len(good_matches)

# 1. 웹캠 Stream을 실행한다.
cap = cv2.VideoCapture(0)

# Load our image template, this is our reference image
# 2. template를 불러온다. 
image_template = cv2.imread('images/box_in_scene.png', 0) 

while True:

    # 3. Get webcam images
    ret, frame = cap.read()

    # 4. Get height and width of webcam frame
    height, width = frame.shape[:2]

    # 4.1 Define ROI Box Dimensions. 
    top_left_x = width / 3
    top_left_y = (height / 2) + (height / 4)
    bottom_right_x = (width / 3) * 2
    bottom_right_y = (height / 2) - (height / 4)
    
    # 4.2 Draw rectangular window for our region of interest   
    cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)
    
    # 5. Crop window of observation we defined above. 이 위치를 가져와서 template과 비교할 예정.
    cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]
    
    # 6. Flip frame orientation horizontally. 거울모드로 돌려놓은 거라고 보면 된다. 이게 more natural하다고.
    frame = cv2.flip(frame,1)
    
    # 7. Get number of SIFT matches. cropped 이미지와 image template를 받아서 match 작업을 진행함.
    matches = sift_detector(cropped, image_template)

    # 8. Display status string showing the current no. of matches. 얼마나 많은 match / key points가 detected되는지.
    # image와 target image 사이의.
    cv2.putText(frame,str(matches),(450,450), cv2.FONT_HERSHEY_COMPLEX, 2,(0,255,0),1)
    
    # 9. Our threshold to indicate object deteciton
    # We use 10 since the SIFT detector returns little false positves. 
    # 10으로 설정했다는 건, 10개 이상 match가 감지될 시 'object detected'로 인식하는 것.
    # 그게 바로 아래 if문이다.
    threshold = 10
    
    # If matches exceed our threshold then object has been detected
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
    
    cv2.imshow('Object Detector using SIFT', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()   

#### Flannbased matching is quite fast, but not the most accurate. Other matching methods include:

BruteForce... Flannbased보다는 accurate하다고.

- BruteForce
- BruteForce-SL2 (not in the documentation, BUT this is the one that skeeps the squared root !)
- BruteForce-L1
- BruteForce-Hamming
- BruteForce-Hamming(2)


## ORB 사용하기.

그런데, ORB를 써서 결과를 내보면 위의 SIFT보다는 결과가 좋지 않다. 잘못된 이미지를 detected표시할 수 있음.

& 단순 이미지를 떠나서, 만약 rotation of image같은 걸 detect하고 싶다면 CNN 형태의 solution이 필요할 수 있다. ORB 대신.

https://www.cs.toronto.edu/~guerzhoy/oriviz/crv17.pdf



In [1]:
import cv2
import numpy as np


def ORB_detector(new_image, image_template):
    # Function that compares input image to template
    # It then returns the number of ORB matches between them
    
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Create ORB detector with 1000 keypoints with a scaling pyramid factor of 1.2
    orb = cv2.ORB_create(1000)

    # Detect keypoints of original image
    # None는 image mask를 쓸지 말지 결정하는 파라미터 부분. 안 쓸 거라 None.
    (kp1, des1) = orb.detectAndCompute(image1, None)
#     print(des1)
    # Detect keypoints of template image
    (kp2, des2) = orb.detectAndCompute(image_template, None)
#     print(des2)
    # Create matcher 
    # Note we're no longer using Flannbased matching
    # Norm_hamming은 distance matric이며, crossCheck은 True로 설정.
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Do matching
    matches = bf.match(des1,des2)

    # Sort the matches based on distance.  Least distance
    # is better
    # 정렬할 수 있음을 보여주려고 sorted를 쓴 거고, 실제로는 large distance를 버리면 성능이 올라간다고 함.
    matches = sorted(matches, key=lambda val: val.distance)

    return len(matches)

cap = cv2.VideoCapture(0)

# Load our image template, this is our reference image
image_template = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/box_in_scene.png', 0) 
# image_template = cv2.imread('images/kitkat.jpg', 0) 

while True:

    # Get webcam images
    ret, frame = cap.read()
    
    # Get height and width of webcam frame
    height, width = frame.shape[:2]

    # Define ROI Box Dimensions (Note some of these things should be outside the loop)
    top_left_x = int(width / 3)
    top_left_y = int((height / 2) + (height / 4))
    bottom_right_x = int((width / 3) * 2)
    bottom_right_y = int((height / 2) - (height / 4))
    
    top = (top_left_x, top_left_y)
    bottom = (bottom_right_x, bottom_right_y)
    # Draw rectangular window for our region of interest
    cv2.rectangle(frame, top, bottom, 255, 3)
    
    # Crop window of observation we defined above
    cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

    # Flip frame orientation horizontally
    frame = cv2.flip(frame,1)
    
    # Get number of ORB matches 
    matches = ORB_detector(cropped, image_template)
    
    # Display status string showing the current no. of matches 
    output_string = "Matches = " + str(matches)
    cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)
    
    # Our threshold to indicate object deteciton
    # For new images or lightening conditions you may need to experiment a bit 
    # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
    threshold = 350
    
    # If matches exceed our threshold then object has been detected
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
    
    cv2.imshow('Object Detector using ORB', frame)
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()   

[[117 241  25  46  24  77  71 216   2 224 169  12 214 213  65  48 152 221
   92 114  75  35 250  57 249 213   5  60  72  59   2 112]
 [121 249  25 100  80 207  71 252  35 224 168  12 115 215  65  48 156 252
   84 110  73  58 216  61 251 205   5  56 104 178  66  56]
 [136 159 123  89 114 232 185  55 124  13 126 242 185  39 164  49  46  58
  134   8 178 184  36 127  70  47  50 193 119 128 101 233]
 [ 27 221  82 188 211 150  30 119  61  72 248 199  44 190  38 212 238 171
  151 164 109 155  16 239 178 246  10 131  59  68 169  95]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[114  33 130 ...  62  43 142]
 [ 11 183 249 ... 198 131 241]
 [ 60  85 123 ... 251 108 115]
 ...
 [216 144 190 ...   1   5  83]
 [171  77  32 ... 221 248 244]
 [ 27 157  82 ...  86 169 223]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46

[[  9 142 215 ... 192 107 133]
 [234 109 246 ... 244 235 138]
 [  1  46 157 ... 224  67 169]
 ...
 [171  92  36 ... 221 248 246]
 [ 56 139 126 ...   0 236 202]
 [ 26 158  66 ... 196 169 155]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 73   5 217 ... 250   1 169]
 [ 72 128  90 ... 162  33  33]
 [129 174 205 ... 224  83 137]
 ...
 [ 56 139 126 ... 128 236 202]
 [218  43 240 ...  12 135 171]
 [ 26 158  82 ... 196 169 155]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[  1 174 222 ... 200 203 133]
 [129 174 207 ... 224  83 169]
 [ 79 182 247 ... 196 167 247]
 ...
 [220  80 191 ...   3  37 115]
 [ 43  77  36 ... 221 248 246]
 [ 27  93   3 ...  86 169 223]]
[[ 56  73 120 ... 171 118 163]
 [162  32

[[118  40 150 ...  63 107 134]
 [143 220 252 ... 252 241 241]
 [  1 175 215 ... 128 193 169]
 ...
 [ 83 207   2 ...  86  57 140]
 [220  80 191 ...   1   5 115]
 [ 27 157  83 ...  86 171 223]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[198  14 199 ... 249  49 133]
 [216 157 182 ... 228 167 123]
 [143 189 242 ... 252 211 241]
 ...
 [222  80 190 ...   1  37  83]
 [ 56 155 126 ...   0 174 202]
 [ 26 159  82 ... 196 169 219]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 63 181 176 ...  79 241 134]
 [143 189 250 ... 236 211 245]
 [143 212 108 ... 252 209 241]
 ...
 [ 56 139 126 ...   4 238 202]
 [ 67 253  35 ...   4 181  92]
 [ 26 159  82 ... 196 169 219]]
[[ 56  73 120 ... 171 118 163]
 [162  32

[[254 124 115 ... 255  39 247]
 [167 238 196 ... 125 218 140]
 [139 191 123 ... 172 211 253]
 ...
 [154 207  18 ...  84  47 142]
 [170  93  61 ...  92 212 250]
 [ 26  31 114 ... 196 169 155]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 41 225  97 ... 134 210 252]
 [ 41 241  97 ... 138 194 244]
 [139 181 248 ... 254 223 253]
 ...
 [170  29 189 ...  92 212 234]
 [ 14 217 191 ...  93 245 246]
 [ 26  31  82 ... 196 169 155]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 47 161 193 ... 239 218 132]
 [143 252 110 ... 252 253 245]
 [143 158 255 ... 245 161 249]
 ...
 [220  80  63 ...   3  37 115]
 [ 39 150 107 ... 139 112 117]
 [ 26  29  82 ... 212 169 155]]
[[ 56  73 120 ... 171 118 163]
 [162  32

[[ 96 164 184 ...  66 198  42]
 [250 237 150 ... 124 169  14]
 [139 149 244 ... 246 211 251]
 ...
 [ 46 249 171 ...  93 245 246]
 [ 58 143  26 ...   4 175 206]
 [ 31  82 226 ... 159 120  85]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[  5  31 215 ... 251 225 177]
 [ 47 161 193 ... 135 210 180]
 [ 61 223 105 ... 187 112 241]
 ...
 [204  16 187 ...   1   5 115]
 [ 58 143  26 ...   4 174 206]
 [ 23  18 226 ... 143 120  85]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[195 191 173 ...  76 214  95]
 [ 95 124 242 ... 255 103 247]
 [ 61 223 105 ... 187 112 241]
 ...
 [170 249  57 ...  93 213 251]
 [204  80 191 ...   1 133 115]
 [ 14 217  59 ...  29 181 242]]
[[ 56  73 120 ... 171 118 163]
 [162  32

...

[[ 32 165 177 ...  66 198 234]
 [193 157 158 ...  64 163  10]
 [ 45 229 169 ... 251 210 160]
 ...
 [191  85 114 ... 215 238 247]
 [142 160 184 ...  13 175 198]
 [ 88  80 123 ... 147  37 123]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[108  37 245 ... 251  70  32]
 [143 243 119 ... 148 175  85]
 [ 40 228 177 ...  82 198 170]
 ...
 [191  85 115 ... 215 238 247]
 [216  16  63 ... 129   5 115]
 [173  84 109 ... 184  82 241]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[158 242 115 ...  22 175 117]
 [240   4  13 ... 137 196   0]
 [ 90  59 122 ... 167  39 155]
 ...
 [171 114  98 ...  12  90 199]
 [191  85 114 ... 215 238 247]
 [216  16  63 ... 131   5 115]]
[[ 56  73 120 ... 171 118 163]
 [162  32

[[236  41 245 ... 235  70  34]
 [201 157  22 ... 112  35 138]
 [ 94 249 251 ... 178  39  59]
 ...
 [142 176 186 ...  13 175 198]
 [208  18  63 ... 131  37 115]
 [141 251 209 ... 248 195 169]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 29 117  99 ... 242 226 241]
 [201 157  22 ... 112 163 168]
 [ 62 114 114 ... 191 102 247]
 ...
 [191  85 115 ... 215 238 247]
 [142 160 184 ...  13 175 198]
 [208  18  63 ... 131  37 115]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 74 182 244 ... 140 231 255]
 [ 92  80 145 ... 122  68  49]
 [ 90 253 178 ... 254 127 219]
 ...
 [191  85 115 ... 215 238 247]
 [174 160 184 ...  13 175 198]
 [208  18  63 ... 129   5 115]]
[[ 56  73 120 ... 171 118 163]
 [162  32

[[ 96 244 177 ...  82 198  34]
 [193 157  23 ...  64 163  10]
 [129 142 215 ... 224 161  33]
 ...
 [ 62  89 115 ... 151 239 247]
 [142 176 186 ...  13 175 198]
 [216  48  61 ...   1   5 115]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[124 108 253 ... 243  70  42]
 [  9 119 107 ... 242 163 177]
 [193 149 150 ...  64 163  11]
 ...
 [142 176 186 ...  13 175 198]
 [216  16  59 ... 131  37 115]
 [192 156 188 ...  88 149  75]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[  9 247 107 ... 242 163 177]
 [142 243  99 ...   6 175 213]
 [193 157  22 ...  64  35  11]
 ...
 [ 62  81 115 ... 215 239 247]
 [142 176 186 ...  13 175 198]
 [216  16  59 ... 131  37 115]]
[[ 56  73 120 ... 171 118 163]
 [162  32

[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 79 251 121 ... 151  11 125]
 [ 86 142  44 ...  69 181  67]
 [143 159 127 ... 206 165 125]
 ...
 [ 14 176 186 ...   7 175 214]
 [ 58 206 110 ... 140 124 223]
 [ 10 148  51 ...  14 181 254]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 74 221 182 ...  92 165  94]
 [ 28  93 122 ... 250 102 219]
 [228 164  21 ...  11 194 130]
 ...
 [210  18 190 ...   1   5 115]
 [ 58 202 110 ... 140 252 202]
 [ 10 148  51 ...  14 181 223]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[162 168  49 ...  76 207 206]
 [193 221

[[124  99 104 ... 187  86   2]
 [116 227   0 ...  11  86 130]
 [100 237 153 ...  91 198  42]
 ...
 [208  90  59 ... 131  37 115]
 [136 159 122 ... 128  36 251]
 [220  16 190 ...   1  37  83]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 92 170 152 ... 192 143  19]
 [248 191  28 ...  72 206  11]
 [ 70 142 174 ... 212 169  65]
 ...
 [128 158 123 ... 136 101 233]
 [ 55  87 106 ... 219 244 243]
 [174 253 185 ...  93 215 251]]
[[ 56  73 120 ... 171 118 163]
 [162  32 102 ... 132  66 211]
 [202  16 235 ...  46 117 251]
 ...
 [124 233 172 ... 115 198  98]
 [156   4  30 ... 241 165  34]
 [ 52 165 121 ... 203  86 226]]
[[ 44 181 177 ... 203 198 173]
 [ 77 181 145 ... 113 135  33]
 [253  80 255 ... 243  70 121]
 ...
 [ 63 127 122 ... 215 238 247]
 [ 51 119 122 ... 155 245 211]
 [220  80  62 ...  19  37 115]]
[[ 56  73 120 ... 171 118 163]
 [162  32